In [ ]:
import pandas as pd
import sqlite3
import requests as re
from lxml import html
import urllib.parse
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from random import randint

In [ ]:
# import the data relate to climbing difficult grades
data = sqlite3.connect("dataset/database.sqlite")
data.row_factory = sqlite3.Row
query = data.execute("SELECT * From grade")
row = query.fetchone()
names = row.keys()
grade_results = pd.DataFrame.from_records(data = query.fetchall(), columns = names)

In [ ]:
# import the data relate to the climbing ascents
data = sqlite3.connect("dataset/database.sqlite")
data.row_factory = sqlite3.Row
query = data.execute("SELECT * From ascent")
row = query.fetchone()
names = row.keys()
ascents_results = pd.DataFrame.from_records(data = query.fetchall(), columns = names)

In [ ]:
# get some infos from the dataset
ascents_results.info()

In [ ]:
grade_results.info()

In [ ]:
# identify the columns
ascents_results.columns

In [ ]:
grade_results.columns

In [ ]:
# remove the nameless rows
df = ascents_results[ascents_results['name'].str.contains('\?',regex=True)]
ascents_results.drop(df.index, inplace=True)

In [ ]:
# remove columns that will not be used
drop_cols = ['user_id','total_score', 'date', 'year', 'last_year', 'rec_date','project_ascent_date','crag_id','sector_id', 'sector','comment', 'rating', 'description', 'yellow_id', 'climb_try','repeat', 'exclude_from_ranking', 'user_recommended', 'chipped','raw_notes','method_id','notes']
ascents_results.drop(drop_cols, axis=1,inplace=True)

In [ ]:
# remove repeated rows by name
ascents_results.drop_duplicates('name',inplace=True)

In [ ]:
# creat a DataFrame for web scrapping the name of location where the ascents were made
crags = ascents_results['crag'].unique()
crags = pd.DataFrame(crags,columns=['crags'])
crags.insert(1,'city','')

In [ ]:
# transform all the string in lower case for web scrapping
crags = crags.apply(lambda x: x.astype(str).str.lower())

In [ ]:
# remove duplicate name os crags
crags.drop_duplicates(subset='crags',keep='last', inplace=True)
crags = crags.reset_index(drop=True)

In [ ]:
# web scrapping of locations in a French climbing site
url = 'https://climbingaway.fr/en/results?search='
for crag in crags['crags']:
    lista = {}
    lista['crag'] = crag
    page = re.get(str(url+urllib.parse.quote(crag)))
    r = html.fromstring(page.content)
    try:
        lista['location'] = r.xpath('/html/body/div[1]/div[2]/div/div/div[1]/div[1]/div[1]/span/text()[1]')
    except:
        lista['location'] = ''
    listas.append(lista)

city_result = pd.DataFrame(listas)

In [ ]:
# identify and remove the elements which the location were not found by web scrapping
empty = city_result[city_result['location']=='[]']
city_result.drop(empty.index, inplace=True)
city_result.reset_index(drop=True, inplace=True)

In [ ]:
# the string returnerd by web scrapping is a entire location from country to city, or province, thisfor will split and extract the city or province
res_list = []
for i in city_result['location']:
    try:
        aux = i.split(', ')[2:4]
    except:
        aux =[]
    res_list.append(aux)
city_result['city'] = np.array(res_list,dtype=object)

In [ ]:
# some strings does not bring an entire location, so the columns city remains empty due the try and except executed before. This will clean the empty results
aux = city_result[city_result['city'] == '[]']
city_result.drop(aux.index, inplace=True)
city_result.reset_index(drop=True, inplace=True)

In [ ]:
# clear and remove special characters from the location and city strings
city_result['location'] = city_result['location'].replace("'",'', regex=True)
city_result['location'] = city_result['location'].replace("\[",'', regex=True)
city_result['location'] = city_result['location'].replace("\]",'', regex=True)
city_result['location'] = city_result['location'].replace('"','', regex=True)
city_result['city'] = city_result['city'].replace("'",'', regex=True)
city_result['city'] = city_result['city'].replace("\[",'', regex=True)
city_result['city'] = city_result['city'].replace("\]",'', regex=True)
city_result['city'] = city_result['city'].replace('"','', regex=True)

In [ ]:
# get latitude and longitude using GeoPy and city columns
aux = []
for i in df['city']:
    info = {}
    info['city'] = i
    try:
        local = Nominatim(user_agent='climb_study').geocode(i)
        info['lat'] = local.latitude
        info['long']  = local.longitude
    except:
        info['lat'] =''
        info['long'] = ''
    aux.append(info)
location = pd.DataFrame(aux)

In [ ]:
# get latitude and longitude using GeoPy and location columns, to obtain tha maximum quantify of locations
aux = []
for i in df['location']:
    info = {}
    info['location'] = i
    try:
        local = Nominatim(user_agent='climb_study').geocode(i)
        info['lat'] = local.latitude
        info['long']  = local.longitude
    except:
        info['lat'] =''
        info['long'] = ''
    aux.append(info)
    
location2 = pd.DataFrame(aux)

In [ ]:
# clear and remove duplicate cities and location
location.drop_duplicates(subset='city',inplace=True)
location.reset_index()
location2.drop_duplicates(subset='location',inplace=True)
location2.reset_index()

In [ ]:
# merge the dataframe os citys with lat and long with the original dataset os ascents
result_cities = pd.merge(city_result,location, how='left', left_on='city',right_on='city')
result_locations = pd.merge(city_result,location2, how='left', left_on='location',right_on='location')

In [ ]:
# create auxiliar dataframes to drop the rows with null values
aux_city = result_cities[result_cities['lat'].isnull()==True]
aux_location = result_locations[result_locations['lat'].isnull()==True]

result_cities.drop(aux_city.index,inplace=True)
result_locations.drop(aux_location.index, inplace=True)

In [ ]:
# append result dataframes
crag_location = result_cities.append(result_locations)
crag_location.drop_duplicates(subset='crag',inplace=True)
crag_location.reset_index(inplace=True)

In [ ]:
# merge the result of location and fill new columns of original ascents results
final_results = pd.merge(ascents_results,crag_location, how='left', left_on='crag',right_on='crag')

In [ ]:
# dataframe with a list of all the result which does not match with any crag in dataframe of location
empty_route = route_location[route_location['lat'].isnull()==True]

In [ ]:
# remove the empty rows
final_ascents_results = final_results.drop(empty_route.index)

In [ ]:
# remove in column and reset index
final_ascents_results.drop(columns='id',inplace=True)
final_ascents_results.reset_index(inplace=True)

In [ ]:
# remove unecessary columns
grade_drop=['score','fra_routes_input','fra_routes_selector','fra_boulders','fra_boulders_input','fra_boulders_selector','usa_routes','usa_routes_input','usa_routes_selector','usa_boulders_input','usa_boulders_selector']
grade.drop(grade_drop,axis=1,inplace=True)

In [ ]:
# merge the ascent dataframe with the grade dataframes
final_ascents_results = pd.merge(final_results,grade, how='left', left_on='grade_id',right_on='id')

In [ ]:
# create an auxiliar dataframe to filter the grade from different types of climbing
lista = []
count=0
for i in final_results['climb_type']:
    grade ={}
    if i==0:
        grade['grade_route'] = final_results['fra_routes'][count]
        grade['grade_boulder'] = ''
    else:
        grade['grade_route'] = ''
        grade['grade_boulder'] = final_results['usa_boulders'][count]
    lista.append(grade)
    count+=1
aux = pd.DataFrame(lista)

In [ ]:
# concatenate the new dataframe with the final dataframe
final_ascents_results = pd.concat([final_ascents_results, aux.reindex(final_results.index)], axis=1)

In [ ]:
# remove the first columns of grade without filter
final_ascents_results.drop(columns=['fra_routes','usa_boulders','id'],inplace=True)

In [ ]:
# export the dataframe
final_ascents_results.to_csv('final_climbing_dataset.csv',encoding='utf-16',index=False)

In [ ]:
# selecting columns to remove from grade_results
drop = ['score','fra_routes_input','fra_routes_selector','fra_boulders','fra_boulders_input','fra_boulders_selector','usa_routes','usa_routes_input','usa_routes_selector','usa_boulders_input','usa_boulders_selector']

In [ ]:
# removing the columns from the dataframe
grade_results.drop(drop,axis=1, inplace=True)

In [ ]:
# saving the dataset
grade_results.to_csv('final_grades_results.csv',encoding='utf-16',index=False)